# TMA4320 - Mal for kode til Prosjekt 1
Dette er et forslag til hvordan du kan strukturere python-kode for å gjøre uavhengig komponentanalyse på lydsignaler.

## Forberedelser
Først må vi importere data. Vi antar i koden nedenfor at lydfilene med miksede signaler ligger på audio/mix_1.wav,
audio/mix_2.wav, audio/mix_3.wav. Du skal i arkivet Prosjekt1.zip kunne finne disse filene samt wav_file_loader.py som blir importert i koden nedenfor.


In [85]:
"""I denne cella laster vi opp tre miksede lydklipp som vi skal bruke å teste ut algoritmen på"""
import numpy as np
from wav_file_loader import read_wavefiles

paths = ['audio/mix_1.wav', 'audio/mix_2.wav', 'audio/mix_3.wav']
data, sampling_rate = read_wavefiles(paths)
num_signals = data.shape[0]

In [86]:
"""I denne cellen fins en funksjon som normaliserer lydsignalenes volum, slik at de får omtrent samme lydstyrke"""
def normalize_audio(data):
    """Scale amplitude s.t. max(data[i]) == 1."""
    abs_data = np.absolute(data)
    maximums = np.amax(abs_data,1)
    # Divide each row by a different vector element:
    data = data / maximums.reshape((3,1))
    return data

data = normalize_audio(data)

In [87]:
"""Her kan du spille av de tre opplastede lydklippene"""
import IPython.display as ipd

ipd.display(ipd.Audio(data[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[2,:], rate=sampling_rate))

## Miksing
Denne delen om miksing, inkludert kodecellen nedenfor kan du foreløpig ignorere. Men dersom du seinere vil teste ut algoritmen ved å selv blande opplastede uavhengige signaler så kan funksjonene nedenfor komme til nytte.

In [88]:
def normalize_rowsums(A):
    """Divide each row in A by its sum.
    
    The sum of each row in the result is 1.0."""
    the_sum = np.sum(A,1)
    A = A / the_sum.reshape((3,1))
    return A

def random_mixing_matrix(signals, observations):
    """ Creates a random matrix
    Each element is a small positive number, not too close to 0.
    (1/11, 5/7).
    """
    
    A = 0.25 + np.random.rand(observations, signals)
    return normalize_rowsums(A)


In [89]:
A = random_mixing_matrix(num_signals, num_signals)
data_mixed = normalize_audio(A @ data)

In [90]:
import IPython.display as ipd

ipd.display(ipd.Audio(data_mixed[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed[2,:], rate=sampling_rate))

## Preprosessering
I den etterfølgende cellen skal du skrive funksjonene <font color='blue'> center_rows </font> og <font color='blue'> whiten_rows </font> for å gjøre de preprosesseringsstegene som er omtalt i prosjektbeskrivelsen. I hvert tilfelle er variabelen Z et array av dimensjon $d\times N$ av miksede signaler.

In [91]:
def center_rows(Z):
    """
    Ensures each row has zero mean.
    Takes a matrix of arbitrary shape and subtracts from each row the mean value of that row.
    """
    
    # Here goes your code. The code should return a dxN-matrix, say Zc, where each row has zero mean
    row_means = np.mean(Z, axis=1)
    Z_transposed = Z.transpose()
    Zc_transposed = Z_transposed - row_means
    Zc = Zc_transposed.transpose()
    return Zc #, mus


def whiten_rows(Z):
    """
    Return whitened version of Z and the matrix for the transform, say Zw, T, where Zw=T*Z
    """
    
    # Covariance matrix, C
    C = np.cov(Z)
    
    # The following two statements compute T (inverse square root of C).
    U, S, _ = np.linalg.svd(C, full_matrices=False)
    T  = U @ np.diag(1 / np.sqrt(S)) @ U.T
    
    # Finally the withened version of Z, Zw
    Zw = np.matmul(T,Z)
    
    return Zw, T

In [92]:
#Test above functions

#Test of center_rows:
Z = np.array([[1,0,1,0],[0,1,0,1],[3,0,1,-3]]) #just an arbitrary test matrix
Zc = center_rows(Z)

print('Zc: \n', Zc, '\n\n')
print('Vector with mean of rows in Zc: \n', np.mean(Zc, axis=1), '\n')

#Test of whiten_rows:
Zcw, T = whiten_rows(Zc)

print('Zwc: \n', Zcw, '\n\n')
print('T: \n', T)

Z_d_w, T = whiten_rows(data)
print(Z_d_w.shape)

Zc: 
 [[ 0.5  -0.5   0.5  -0.5 ]
 [-0.5   0.5  -0.5   0.5 ]
 [ 2.75 -0.25  0.75 -3.25]] 


Vector with mean of rows in Zc: 
 [0. 0. 0.] 

Zwc: 
 [[ 0.14106542 -0.93597885  0.77699617  0.01791727]
 [-0.14106539  0.9359789  -0.7769962  -0.01791731]
 [ 1.0824692   0.19065315  0.06397132 -1.33709367]] 


T: 
 [[ 4.33800115e+07  4.33800095e+07 -3.17965374e-01]
 [ 4.33800095e+07  4.33800115e+07  3.17965405e-01]
 [-3.17965374e-01  3.17965405e-01  5.09248941e-01]]
(3, 50000)


## Hovediterasjonen - maksimering av ikke-gaussiskhet

In [93]:
def normalize_rownorms(Z):
    """
    Divide each row in A by its Euclidean norm.
    
    The norm of each row in the output is equal to one.
    
    Computes the Euclidean norm of each row of the matrix Z
    and then scale each row by this norm. 
    """
     
    e_norms = np.linalg.norm(Z, axis=1)
    Z_trans = Z.transpose()
    Z_norm_trans = Z_trans / e_norms
    Z_norm = Z_norm_trans.transpose()
    return Z_norm

In [94]:
#Test og normalize_rownorms:
Z_norm = normalize_rownorms(Z)

print('Z_norm: \n',Z_norm, '\n')
print('Vector with row norms: \n', np.linalg.norm(Z_norm, axis=1))

Z_norm: 
 [[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.6882472   0.          0.22941573 -0.6882472 ]] 

Vector with row norms: 
 [1. 1. 1.]


In [95]:
def decorrelate_weights(W):
    """
    This is the orthogonalization step (or decorrelation step) The dxd input matrix W is projected onto an 
    orthogonal matrix by the transformation Wd = (WW^T)^{-1/2} W as described in the note. The single output 
    argument is the projected W-matrix (Wd).
    
    Uses a similar technque for computing the inverse square root as in the whitening step.
    """
    
    # Matrix product of W and its transposed
    WW_T = np.matmul(W,W.transpose())
    
    # Computes T, the inverse square root of WW_T
    U, S, _ = np.linalg.svd(WW_T, full_matrices=False)
    T = U @ np.diag(1 / np.sqrt(S)) @ U.transpose()
    
    # Finally computes the decorrelated matrix Wd
    Wd = np.matmul(T, W)

    return Wd


In [96]:
#Test of decorrelate_weights:
W = np.array([[1,4,5],[6,1,3],[4,9,5]]) # An arbitrart dxd matrix
Wd = decorrelate_weights(W)

print('Wd: \n', Wd, '\n\n')
print(np.sum(Wd, axis=0), '\n')
print('cov-matrix of Wd: \n', np.cov(Wd), '\n')
print('cov-matrix of W: \n', np.cov(W))

Wd: 
 [[-0.27872392  0.06016608  0.95848475]
 [ 0.91996083 -0.26973763  0.2844533 ]
 [ 0.27565385  0.96105237  0.01983189]] 


[0.91689075 0.75148082 1.26276994] 

cov-matrix of Wd: 
 [[ 0.40875136 -0.11526538 -0.15495773]
 [-0.11526538  0.35439664 -0.19574277]
 [-0.15495773 -0.19574277  0.236852  ]] 

cov-matrix of W: 
 [[ 4.33333333 -4.16666667  2.5       ]
 [-4.16666667  6.33333333 -6.        ]
 [ 2.5        -6.          7.        ]]


In [97]:
def update_W(W, Zcw):
    """x
    Calculates W_k+1 from W_k.
    The input is W=W_k (d x d) as well as the cenztered, whitened data Zcw (dxN known as tilde{x} in the note)
    Output is the new W (W_{k+1}).
    
    This function does the two iteration steps in the note: The optimisation step and the 
    orthogonalisation (decorrelation) step. The first step, the orthogonalisation is provided by the 
    function decorrelate_weights which is called below.
    
    Uses kurtosis G(u) = 4*(u^3)
    """
    
    # Optimization step:
    s_k = np.dot(W, Zcw)
    
    # kurtosis and derivative as lambda funtions
    kurtosis = lambda u: 4*(u**3)
    kurtosis_d = lambda u: 12*(u**2)
    
    # Applies kurtosis function and derivative on each element of s_k
    G = kurtosis(s_k)
    G_d = kurtosis_d(s_k)
    
    # Gets number of rows and columns of G_d
    N = G_d.shape[1]
    
    # Computes W_k+1, here called W_p
    W_p = (1/N) * np.dot(G, Zcw.transpose()) - np.matmul(np.diag(np.average(G_d, axis=1)), W)
    
    # Normalizes rows of W_p (W_k+1)
    W_pn = normalize_rownorms(W_p)
    
    #########################################################################################
    
    # Orthogonalization step:
    W_pnd = decorrelate_weights(W_pn)

    return W_pnd
    

In [98]:
W_p = update_W(W, Zcw)

print(W_p)

[[ 0.21195709 -0.42398307 -0.88051834]
 [-0.93141629  0.18513155 -0.31335285]
 [-0.29586802 -0.88654649  0.35566479]]


In [99]:
def measure_of_convergence(W1, W2):
    """
    This function computes an error estimate for the maximisation iteration, it computes the convergence
    criterion given in the note. 
    Input: W1 is the previous iterate, and W2 is the one just computed.
    Output: The quantity delta defined in the note.
    Typical numpy-functions to use: numpy.sum, numpy.absolute, numpy.amax.
    """
    
    a_s = np.absolute(np.sum(np.multiply(W2, W1), axis=1))
    delta = np.absolute(np.amax((1-a_s)))
    return delta

In [100]:
import numpy as np
W1 = np.array([[4,5,9],[80,2,5],[2,2,1]])
W2 = np.array([[6,3,7], [9,3,6], [5,3,1]])
print(measure_of_convergence(W1, W2))
print(np.multiply(W2, W1))
print(np.sum(np.multiply(W2, W1), axis = 1))

16
[[ 24  15  63]
 [720   6  30]
 [ 10   6   1]]
[102 756  17]


In [101]:
import warnings

tol_default = 1e-10

def fast_ICA(Z, signals_to_find, tol=tol_default, max_iter=100):
    """ This is the function that organises all the work.
    
    Input: Z is the unprocessed data
           signals_to_find: in our case, always d the number of sources
           tol is the tolerance, default value 1.0e-10
           max_iter abort after max_iter iterations if not converged, (to avoid infinite loop)
    Output: Z_ica, the separated signals (dxN matrix, approximating the sources)
            W The final converged W-matrix (dxd)
            Also some other variables of interest can be returned if desired
    """
    
    # center the rows of Z
    Z_cent = center_rows(Z)

    # whiten the centered rows
    Z_cent_wit, T = whiten_rows(Z_cent)
    
    # Put W_0 = W to a random initial value and normalise the rows to length 1
    M = Z_cent.shape[0]
    W_0 = np.random.rand(M, M)
    W_0 = normalize_rowsums(W_0)
    
    # Initialise some variables to prepare for the while-loop (such as delta)
    delta = tol + 1
    number_of_iter = 0

    # while delta>tol and number_of_iterations < max_iter:
    #      do an iteration to get a new W-iterate 
    #      Compute the error estimate to update delta

    while delta > tol and number_of_iter < max_iter:
        W_p = update_W(W_0, Z_cent_wit)
        delta = measure_of_convergence(W_0, W_p)
        W_0 = W_p
        number_of_iter += 1
        print("Delta= ", delta)
        
    # Clean up, check if converged or max_iter attained
    if number_of_iter == max_iter:
        print('max_iter reached :(((\ndelta: ', delta)
        
    else:
        print("Delta= ", delta)
        
    Z_ica = np.matmul(W_p, Z_cent_wit)
    return Z_ica, W_p
    


In [102]:
Z_ica, W_ica = fast_ICA(data, 3)
print(W_ica.shape)
print(Z_ica.shape)

Delta=  0.6958781965236789
Delta=  0.10242369576782995
Delta=  0.018269092623930838
Delta=  0.026278934014787314
Delta=  0.02736855811393546
Delta=  0.0025868648082115753
Delta=  5.410400641547852e-05
Delta=  1.038008704101756e-06
Delta=  2.0313682957961987e-08
Delta=  3.9218595038192916e-10
Delta=  7.710054816811862e-12
Delta=  7.710054816811862e-12
(3, 3)
(3, 50000)


In [103]:
"""Her kan du spille av de tre opplastede lydklippene"""
import IPython.display as ipd

ipd.display(ipd.Audio(data[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[2,:], rate=sampling_rate))

In [104]:
"""Her kan du spille av de tre lydklippene etter de har vært igjennom sepparasjonen"""
import IPython.display as ipd

ipd.display(ipd.Audio(Z_ica[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ica[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ica[2,:], rate=sampling_rate))

In [105]:
data

array([[ 0.02745098,  0.35686275,  0.28627451, ...,  0.23137255,
         0.27843137,  0.36470588],
       [-0.06666667,  0.29411765,  0.2627451 , ...,  0.23921569,
         0.28627451,  0.36470588],
       [-0.50588235,  0.15294118,  0.18431373, ...,  0.28627451,
         0.34901961,  0.45098039]])

In [106]:
A = np.identity(3)
update_W(A, data)

array([[-9.99999522e-01, -9.66654418e-04,  1.45041989e-04],
       [ 9.66805154e-04, -9.99998989e-01,  1.04280845e-03],
       [-1.44033807e-04, -1.04294818e-03, -9.99999446e-01]])

In [107]:
A = np.array([[0.7, 0.3, 0],
             [0.1, 0.8, 0],
              [0.4, 0.4, 0.1]])
A = normalize_rownorms(A)
print(A)
print(decorrelate_weights(A))
print(data)
print(update_W(A, data))

[[0.91914503 0.3939193  0.        ]
 [0.12403473 0.99227788 0.        ]
 [0.69631062 0.69631062 0.17407766]]
[[ 0.86941082  0.04836663 -0.49171688]
 [-0.22631035  0.92364684 -0.30928973]
 [ 0.43921344  0.38018045  0.81397443]]
[[ 0.02745098  0.35686275  0.28627451 ...  0.23137255  0.27843137
   0.36470588]
 [-0.06666667  0.29411765  0.2627451  ...  0.23921569  0.28627451
   0.36470588]
 [-0.50588235  0.15294118  0.18431373 ...  0.28627451  0.34901961
   0.45098039]]
[[-0.83716022 -0.01832596  0.54665065]
 [ 0.25441892 -0.89778059  0.35952891]
 [-0.48418363 -0.44006156 -0.75624866]]
